In [26]:
import torch
# the first flag below was False when we tested this script but True makes A100 training a lot faster:
# torch.backends.cuda.matmul.allow_tf32 = True
# torch.backends.cudnn.allow_tf32 = True
# import torch.distributed as dist
# from torch.nn.parallel import DistributedDataParallel as DDP
from torch.utils.data import DataLoader
# from torch.utils.data.distributed import DistributedSampler
from torchvision.datasets import ImageFolder
from torchvision import transforms
import numpy as np
from collections import OrderedDict
from PIL import Image
from copy import deepcopy
from glob import glob
from time import time
import argparse
import logging
import os

from models import SiT_models
from download import find_model
from transport import create_transport, Sampler
from diffusers.models import AutoencoderKL
from torchvision.datasets import CIFAR10

from train_utils import parse_transport_args
import wandb_utils

In [27]:
@torch.no_grad()
def update_ema(ema_model, model, decay=0.9999):
    """
    Step the EMA model towards the current model.
    """
    ema_params = OrderedDict(ema_model.named_parameters())
    model_params = OrderedDict(model.named_parameters())

    for name, param in model_params.items():
        # TODO: Consider applying only to params that require_grad to avoid small numerical changes of pos_embed
        ema_params[name].mul_(decay).add_(param.data, alpha=1 - decay)


In [28]:
def requires_grad(model, flag=True):
    """
    Set requires_grad flag for all parameters in a model.
    """
    for p in model.parameters():
        p.requires_grad = flag

In [29]:
# def create_logger(logging_dir):
#     """
#     Create a logger that writes to a log file and stdout.
#     """
#     if dist.get_rank() == 0:  # real logger
#         logging.basicConfig(
#             level=logging.INFO,
#             format='[\033[34m%(asctime)s\033[0m] %(message)s',
#             datefmt='%Y-%m-%d %H:%M:%S',
#             handlers=[logging.StreamHandler(), logging.FileHandler(f"{logging_dir}/log.txt")]
#         )
#         logger = logging.getLogger(__name__)
#     else:  # dummy logger (does nothing)
#         logger = logging.getLogger(__name__)
#         logger.addHandler(logging.NullHandler())
#     return logger
def create_logger(logging_dir):
    """
    Create a logger that writes to a log file and stdout.
    """
    logging.basicConfig(
        level=logging.INFO,
        format='[\033[34m%(asctime)s\033[0m] %(message)s',
        datefmt='%Y-%m-%d %H:%M:%S',
        handlers=[logging.StreamHandler(), logging.FileHandler(f"{logging_dir}/log.txt")]
    )
    logger = logging.getLogger(__name__)

    return logger

In [30]:
def center_crop_arr(pil_image, image_size):
    """
    Center cropping implementation from ADM.
    https://github.com/openai/guided-diffusion/blob/8fb3ad9197f16bbc40620447b2742e13458d2831/guided_diffusion/image_datasets.py#L126
    """
    while min(*pil_image.size) >= 2 * image_size:
        pil_image = pil_image.resize(
            tuple(x // 2 for x in pil_image.size), resample=Image.BOX
        )

    scale = image_size / min(*pil_image.size)
    pil_image = pil_image.resize(
        tuple(round(x * scale) for x in pil_image.size), resample=Image.BICUBIC
    )

    arr = np.array(pil_image)
    crop_y = (arr.shape[0] - image_size) // 2
    crop_x = (arr.shape[1] - image_size) // 2
    return Image.fromarray(arr[crop_y: crop_y + image_size, crop_x: crop_x + image_size])



In [31]:
results_dir = 'workspace'
model_name = 'SiT-S/4'
image_size = 32
num_classes = 1
ckpt = None
device = 'mps'
data_path = 'data'
os.makedirs(data_path, exist_ok=True)


In [32]:
os.makedirs(results_dir, exist_ok=True)  # Make results folder (holds all experiment subfolders)
experiment_index = len(glob(f"{results_dir}/*"))
model_string_name = model_name.replace("/", "-")  # e.g., SiT-XL/2 --> SiT-XL-2 (for naming folders)

In [33]:
path_type = 'Linear'
prediction = 'velocity'
loss_weight = 1.0
vae = 'ema'
batch_size = 64
cfg_scale = 10.0
epochs = 1000
log_every = 10
use_wandb = True
ckpt_every = 200
sample_every = 50
global_batch_size = 64

In [34]:

experiment_name = f"{experiment_index:03d}-{model_string_name}-" \
                f"{path_type}-{prediction}-{loss_weight}"
experiment_dir = f"{results_dir}/{experiment_name}"  # Create an experiment folder
checkpoint_dir = f"{experiment_dir}/checkpoints"  # Stores saved model checkpoints
os.makedirs(checkpoint_dir, exist_ok=True)
logger = create_logger(experiment_dir)
logger.info(f"Experiment directory created at {experiment_dir}")

entity = 'climate_si_team' #os.environ["ENTITY"]
project = 'Cifar SI'
#construct args
args = argparse.Namespace(
    model_name=model_name,
    image_size=image_size,
    num_classes=num_classes,
    ckpt=ckpt,
    device=device,
    data_path=data_path,
    path_type=path_type,
    prediction=prediction,
    loss_weight=loss_weight,
    vae=vae,
    batch_size=batch_size,
    cfg_scale=cfg_scale,
    epochs=epochs,
    log_every=log_every,
    use_wandb=use_wandb,
    ckpt_every=ckpt_every,
    sample_every=sample_every,
    global_batch_size=global_batch_size
)
if use_wandb:
    wandb_utils.initialize(args=args, entity=entity, exp_name=experiment_name, project_name=project)

[2024-09-29 19:15:27] Experiment directory created at workspace/003-SiT-S-4-Linear-velocity-1.0


train loss,█▃▂▂▂▂▂▂▂▂▁▁▂▂▁▁▁▁▁▁▁▁▁▁▂▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁
train steps/sec,███▁████████████▂██▁██▁█▇▁█▇▇█████▁▇████
train_step,▁▁▁▂▂▂▂▃▃▃▃▃▃▃▄▄▅▅▅▅▅▅▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇██
train loss,0.98497
train steps/sec,9.30184
train_step,13950


In [106]:
 # Create model:
assert image_size % 8 == 0, "Image size must be divisible by 8 (for the VAE encoder)."
latent_size = image_size #// 8
model = SiT_models[model_name](
    input_size=latent_size,
    in_channels=3,
    num_classes=num_classes
)
model = model.to(device)

In [107]:
# Note that parameter initialization is done within the SiT constructor
ema = deepcopy(model).to(device)  # Create an EMA of the model for use after training

if ckpt is not None:
    ckpt_path = ckpt
    state_dict = find_model(ckpt_path)
    model.load_state_dict(state_dict["model"])
    ema.load_state_dict(state_dict["ema"])
    opt.load_state_dict(state_dict["opt"])
    args = state_dict["args"]

requires_grad(ema, False)

In [108]:
transport = create_transport(
        path_type,
        prediction,
        loss_weight,
        # train_eps,
        # sample_eps
    )  # default: velocity; 
transport_sampler = Sampler(transport)
# vae = AutoencoderKL.from_pretrained(f"stabilityai/sd-vae-ft-{vae}").to(device)
logger.info(f"SiT Parameters: {sum(p.numel() for p in model.parameters()):,}")

[2024-09-29 19:42:11] SiT Parameters: 32,542,944


In [109]:
transform = transforms.Compose([
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5], inplace=True)
])

# Load CIFAR-10 dataset
dataset = CIFAR10(root=data_path, train=True, download=True, transform=transform)

# Use DistributedSampler for distributed training
# sampler = DistributedSampler(
#     dataset,
#     num_replicas=dist.get_world_size(),
#     rank=rank,
#     shuffle=True,
#     seed=args.global_seed
# )

# Create DataLoader
loader = DataLoader(
    dataset,
    batch_size=batch_size,
    shuffle=False,
    # num_workers=args.num_workers,
    pin_memory=True,
    drop_last=True
)
logger.info(f"Dataset contains {len(dataset):,} images ({data_path})")

Files already downloaded and verified


[2024-09-29 19:42:16] Dataset contains 50,000 images (data)


In [118]:
# Prepare models for training:
update_ema(ema, model, decay=0)  # Ensure EMA is initialized with synced weights
model.train()  # important! This enables embedding dropout for classifier-free guidance
ema.eval()  # EMA model should always be in eval mode

# Variables for monitoring/logging purposes:
train_steps = 0
log_steps = 0
running_loss = 0
start_time = time()

# Labels to condition the model with (feel free to change):
ys = torch.randint(1000, size=(batch_size,), device=device)
use_cfg = cfg_scale > 1.0
# Create sampling noise:
n = ys.size(0)
# zs = torch.randn(n, 4, latent_size, latent_size, device=device)
zs = torch.randn(n, 3, latent_size, latent_size, device=device)

In [119]:
if use_cfg:
    zs = torch.cat([zs, zs], 0)
    y_null = torch.tensor([1000] * n, device=device)
    ys = torch.cat([ys, y_null], 0)
    sample_model_kwargs = dict(y=ys, cfg_scale=cfg_scale)
    model_fn = ema.forward_with_cfg
else:
    sample_model_kwargs = dict(y=ys)
    model_fn = ema.forward


In [120]:
zs.shape

torch.Size([128, 3, 32, 32])

In [121]:
opt = torch.optim.AdamW(model.parameters(), lr=1e-4, weight_decay=0)

In [122]:
# import PIL 
# from PIL import Image
# 
# #show x
# x, y = next(iter(loader))
# x = x[0]
# x = x.permute(1, 2, 0)
# x = x.cpu().detach().numpy()
# x = (x + 1) / 2
# x = x * 255
# x = x.astype(np.uint8)
# img = Image.fromarray(x)
# img

In [123]:
# for x, y in loader:
#     # print(x.shape)
#     # print(x, y)
#     #show x picture
#     def show_image(x):
#         x = x[0]
#         x = x.permute(1, 2, 0)
#         x = x.cpu().detach().numpy()
#         x = (x + 1) / 2
#         x = x * 255
#         x = x.astype(np.uint8)
#         img = Image.fromarray(x)
#         return img
#     
#     img = show_image(x)
#     latent_img = vae.encode(x.to(device)).latent_dist.sample().mul_(0.18215)
#     # print(latent_img.latent_dist.sample())
#     decoded_img = vae.decode( latent_img / 0.18215 ).sample
#     decoded_image_show = show_image(decoded_img)
#     # print(decoded_img.sample())
#     break

In [124]:
logger.info(f"Training for {epochs} epochs...")
for epoch in range(epochs):
    # sampler.set_epoch(epoch)
    logger.info(f"Beginning epoch {epoch}...")
    for x, y in loader:
        x = x.to(device)
        y = y.to(device)
        # with torch.no_grad():
            # Map input images to latent space + normalize latents:
            # x = vae.encode(x).latent_dist.sample().mul_(0.18215)
        model_kwargs = dict(y=y)
        loss_dict = transport.training_losses(model, x, model_kwargs)
        loss = loss_dict["loss"].mean()
        opt.zero_grad()
        loss.backward()
        opt.step()
        update_ema(ema, model)

        # Log loss values:
        running_loss += loss.item()
        log_steps += 1
        train_steps += 1
        
        if train_steps % log_every == 0:
            # Measure training speed:
            # torch.cuda.synchronize()
            end_time = time()
            steps_per_sec = log_steps / (end_time - start_time)
            # # Reduce loss history over all processes:
            avg_loss = torch.tensor(running_loss / log_steps, device=device)
            # dist.all_reduce(avg_loss, op=dist.ReduceOp.SUM)
            # avg_loss = avg_loss.item() / dist.get_world_size()
            # logger.info(f"(step={train_steps:07d}) Train Loss: {avg_loss:.4f}, Train Steps/Sec: {steps_per_sec:.2f}")
            logger.info(f"(Train Loss: {avg_loss:.4f}, Train Steps/Sec: {steps_per_sec:.2f}")
            
            if use_wandb:
              
                wandb_utils.log(
                    { "train loss": avg_loss, "train steps/sec": steps_per_sec },
                    step=train_steps
                )
            # Reset monitoring variables:
            running_loss = 0
            log_steps = 0
            start_time = time()

        # Save SiT checkpoint:
        if train_steps % ckpt_every == 0 and train_steps > 0:
            # if rank == 0:
            checkpoint = {
                "model": model.state_dict(),
                "ema": ema.state_dict(),
                "opt": opt.state_dict(),
                # "args": args
            }
            checkpoint_path = f"{checkpoint_dir}/{train_steps:07d}.pt"
            torch.save(checkpoint, checkpoint_path)
            logger.info(f"Saved checkpoint to {checkpoint_path}")
            # dist.barrier()
        
        if train_steps % sample_every == 0 and train_steps > 0:
            logger.info("Generating EMA samples...")
            sample_fn = transport_sampler.sample_ode() # default to ode sampling
            samples = sample_fn(zs, model_fn, **sample_model_kwargs)[-1]
            # dist.barrier()

            if use_cfg: #remove null samples
                samples, _ = samples.chunk(2, dim=0)
            # samples = vae.decode(samples / 0.18215).sample
            # out_samples = torch.zeros((global_batch_size, 3, image_size, image_size), device=device)
            out_samples = samples
            
            if use_wandb:
                wandb_utils.log_image(out_samples, train_steps)
            logging.info("Generating EMA samples done.")

model.eval()  # important! This disables randomized embedding dropout
# do any sampling/FID calculation/etc. with ema (or model) in eval mode ...

logger.info("Done!")
    # cleanup()

[2024-09-29 19:44:49] Training for 1000 epochs...
[2024-09-29 19:44:49] Beginning epoch 0...
[2024-09-29 19:44:53] (Train Loss: 0.7534, Train Steps/Sec: 1.11
[2024-09-29 19:44:55] (Train Loss: 0.6199, Train Steps/Sec: 5.94
[2024-09-29 19:44:57] (Train Loss: 0.5110, Train Steps/Sec: 6.01
[2024-09-29 19:44:58] (Train Loss: 0.4768, Train Steps/Sec: 6.05
[2024-09-29 19:45:00] (Train Loss: 0.4293, Train Steps/Sec: 6.03
[2024-09-29 19:45:00] Generating EMA samples...
wandb: WARNING Tried to log to step 10 that is less than the current step 50. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 20 that is less than the current step 50. Steps must be monotonically increasing, so this data will be ignored. See https://wandb.me/define-metric to log data out of order.
wandb: WARNING Tried to log to step 30 that is less than the current step 50. Steps must be monotonically increasing

KeyboardInterrupt: 